<a href="https://colab.research.google.com/github/pinilDissanayaka/SELF-RAG-Agent-/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.5/245.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import os
from typing import List, TypedDict, Dict, Any, Literal

from google.colab import userdata
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_groq.chat_models import ChatGroq
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [5]:
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_PROJECT_ID"]=userdata.get('GOOGLE_PROJECT_ID')

In [6]:
class GraphState(TypedDict):
  message: List
  genaration: List
  retriever: List

In [7]:
pinecone=Pinecone()

index_names=pinecone.list_indexes().names()

print(f"All the indexes at the vectore store : {index_names}")
print(f"Selected index from vectore store : {index_names[-1]}")

index=pinecone.Index(index_names[1])

All the indexes at the vectore store : ['randika', 'llama3']
Selected index from vectore store : llama3


In [8]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [9]:
pinecone_vectorstore=PineconeVectorStore(index=index, embedding=embeddings)

retriever=pinecone_vectorstore.as_retriever()

In [10]:
def retrieve(state:GraphState):
  state['retriever'].append(retriever.invoke(state['message'][-1]))
  return state

In [11]:
def genaration(state:GraphState):
  prompt_template = """Given the following context and a question, generate an answer based on this context only.which can include text, images and tables.
              In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
              If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

              CONTEXT: {context}

              QUESTION: {question}

              response :
            """
  prompt=ChatPromptTemplate.from_template(prompt_template)

  llm=ChatGroq(model="llama3-groq-70b-8192-tool-use-preview", temperature=0.2)

  genaration_chain=prompt | llm | StrOutputParser()

  response=genaration_chain.invoke({"context":state['retriever'][-1], "question":state['message'][-1]})

  state['genaration'].append(response)

  return state

In [14]:
class Grade(BaseModel):
  binary_score:str=Field(description="description of the grade")

In [ ]:
def grader(state:GraphState):
  grading_prompt="""You are a grader assessing relevance of a retrieved document to a user question. \n
                It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
                If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
                Give a binary_score 'yes' or 'no' score to indicate whether the document is relevant to the question."""